In [70]:
import pandas as pd
import datetime
import numpy as np
import scipy.signal
from scipy.stats import skew, kurtosis
import copy
from rhythm_functions import *

In [66]:
# Load the hourly data
file_path = '../Studies/oura_feats_bin01_23_hourly.csv'
hourly_data = pd.read_csv(file_path)

# Convert 'hour' to datetime and set it as the index
hourly_data['hour'] = pd.to_datetime(hourly_data['hour'])
hourly_data.set_index('hour', inplace=True)

# Interpolate missing values linearly
hourly_data.interpolate(method='linear', inplace=True)

# Forward fill and backward fill
hourly_data.fillna(method='ffill', inplace=True)
hourly_data.fillna(method='bfill', inplace=True)

# Identify and remove participants or days with excessive missing data
missing_data_threshold = 0.5  # 50% threshold
grouped_data = hourly_data.groupby([hourly_data.index.date, 'participant_id']).apply(
    lambda group: group.isna().mean()
)
excessive_missing_data = grouped_data[grouped_data > missing_data_threshold].dropna(how='all')

# Handle days with excessive missing data (if any)
hourly_data.reset_index(inplace=True)

# Prepare the final datasets
# Dataset with imputed values
# hourly_data.to_csv('/path/to/amed_clean_hour_template.csv')

# Dataset with missing values imputed as zeros
hour_data_zeros = hourly_data.fillna(0)
# hour_data_zeros.to_csv('/path/to/amed_clean_hour_zeros.csv')


In [85]:
def get_time_window_oura(data, window_length):
    if window_length > 21:
        print('Invalid Length')
        return

    window_l = window_length  # In days
    window_Delta = timedelta(days=window_l)
    grouped = data.groupby(by=['participant_id'])
    # print(grouped.mean())
    data_window = []

    for participant_id, group in grouped:
        
        df = group.copy()
        df['hour'] = pd.to_datetime(df['hour'])
        df.sort_values(by='hour', inplace=True)

        start = df['hour'].iloc[0]
        end = start + window_Delta

        # Selecting data within the window
        data_values = df[(df['hour'] >= start) & (df['hour'] < end)]
        # # Check if the length of data matches the window length
        # if len(data_values) != window_l * 24:
        #     continue

        # Add back the participant_id
        data_values['participant_id'] = participant_id
        data_window.append(data_values)
    
    # Concatenating all participant data into a single DataFrame
    data_window = pd.concat(data_window, ignore_index=True)
    
    return data_window

data7 = get_time_window_oura(hourly_data, 7)
data14 = get_time_window_oura(hourly_data, 14)
data21 = get_time_window_oura(hourly_data, 21)

In [83]:
def M10(data, value_col, hour_col):
    m10_list = []
    for _, day_data in data.groupby(data[hour_col].dt.date):
        m10 = 0
        # if len(day_data) >= 10:  # Only consider days with at least 10 data points
        sorted_day_data = day_data.sort_values(by=hour_col)
        data_array = sorted_day_data[value_col].values
        for i in range(len(data_array)-9):
            m10 = max(m10, sum(data_array[i:i+10])/10)
        m10_list.append(m10)
    return np.nanmean(m10_list) if m10_list else np.nan

def L5(data, value_col, hour_col):
    l5_list = []
    for _, day_data in data.groupby(data[hour_col].dt.date):
        l5 = 0
        # if len(day_data) >= 5:  # Only consider days with at least 5 data points
        sorted_day_data = day_data.sort_values(by=hour_col)
        data_array = sorted_day_data[value_col].values
        for i in range(len(data_array)-4):
            l5 = max(l5, sum(data_array[i:i+5])/5)
        l5_list.append(l5)
    return np.nanmean(l5_list) if l5_list else np.nan

def get_template(data, value_col, hour_col='hour', window_l=5):
    template = []
    for hour in range(24):
        hourly_data = data[data[hour_col].dt.hour == hour]
        if not hourly_data.empty:
            val_mean = hourly_data[value_col].mean()
        else:
            val_mean = np.nan  # No data for this hour
        template.append({'hour': hour, value_col: val_mean})

    return pd.DataFrame(template)



def deviation_from_template(data, value_col, hour_col='hour'):
    template = get_template(data, value_col, hour_col)
    df_result = []

    for date, day_data in data.groupby(data[hour_col].dt.date):
        hourly_diff = []

        for hour in range(24):
            template_val = template.loc[template['hour'] == hour, value_col].values[0]
            if not np.isnan(template_val) and hour in day_data[hour_col].dt.hour.values:
                day_val = day_data[day_data[hour_col].dt.hour == hour][value_col].values[0]
                hourly_diff.append(template_val - day_val)

        if hourly_diff:  # Proceed if there are any non-NaN differences
            df = pd.DataFrame({
                'mean_diff': [np.mean(hourly_diff)],
                'median_diff': [np.median(hourly_diff)],
                'sd_diff': [np.std(hourly_diff)],
                'skew': [skew(hourly_diff)],
                'kurtosis': [kurtosis(hourly_diff)]
            })
            df_result.append(df)

    return pd.concat(df_result, ignore_index=True) if df_result else pd.DataFrame()


def fit_cosinor(df, value_col, hour_col, periods):
    # Copy and prepare the data
    signal = df.copy()
    signal.rename(columns={value_col: 'y'}, inplace=True)
    
    # Convert datetime to numeric (e.g., hours since start)
    start_time = signal[hour_col].min()
    signal['x'] = (signal[hour_col] - start_time).dt.total_seconds() / 3600  # Convert to hours

    variables = {}

    for pi in periods:
        if pi < 8:
            continue

        model, stats, params, xtest, ytest = cosinor.fit_me(signal['x'].values, signal['y'].values, period=pi, plot=False)

        mesor = params['mesor']
        Amp = params['amplitude']
        phi = params['acrophase']
        rss = stats['p_reject']

        variables[value_col + '_' + str(pi) + 'mesor'] = mesor
        variables[value_col + '_' + str(pi) + 'amp'] = Amp
        variables[value_col + '_' + str(pi) + 'phi'] = phi
        variables[value_col + '_' + str(pi) + 'p_reject'] = rss

    return variables

def cal_rhythm_features(data):
    grouped = data.groupby(by=['participant_id'])
    columns = ['avg_hr', 'std_hr', 'Sleep Stage Sum']
    rhythm_feats = []
    for group in grouped:
        
        df = copy.deepcopy(group[1])
        ID = group[0]
        curr_fea_df = []
        
        for col_val in columns:
            curr_window = copy.deepcopy(df[[col_val,'hour']])
            
            M10H = M10(curr_window, col_val, 'hour')
            L5H = L5(curr_window, col_val, 'hour')
            AMP = (M10H-L5H)/(M10H+L5H)
            DVT = deviation_from_template(curr_window, col_val, 'hour')
            
            period_list = [8,16,20, 22, 27, 28, 32, 36, 64, 72, 128]
            PSD = power_spectral_density(curr_window[col_val], period_list)
            
            cosinor_value = fit_cosinor(curr_window, col_val, 'hour', period_list) 
            fea_dict = {'M10':[M10H],'L5':[L5H], 'AMP':[AMP]}
            fea_dict.update(PSD)
            fea_dict.update(cosinor_value)
            fea_dict2 = {col_val+'_'+k: v for k,v in fea_dict.items()}
            DVT = DVT.mean().to_dict()
            fea_df = pd.DataFrame(data={**fea_dict2, **DVT})
            if len(curr_fea_df) == 0:
                curr_fea_df = fea_df
            else:
                curr_fea_df = pd.concat((curr_fea_df,fea_df), axis=1)
        curr_fea_df['ID'] = ID
        
        if len(rhythm_feats) == 0:
            rhythm_feats = curr_fea_df
        else:
            rhythm_feats = pd.concat((rhythm_feats,curr_fea_df), axis=0) 
    rhythm_feats.reset_index(inplace=True, drop=True)
    return rhythm_feats


In [88]:
rhythm_7days = cal_rhythm_features(data7)
rhythm_14days = cal_rhythm_features(data14)
rhythm_21days = cal_rhythm_features(data21)

In [90]:
rhythm_7days.reset_index(inplace=True, drop=True)
feat_type = 'template'
save_path = '../Studies/BIN_features/rhythm_features/'
rhythm_7days.to_csv(save_path + 'rhythm7days'+feat_type+'.csv', index=False)  
rhythm_14days.to_csv(save_path + 'rhythm14days'+feat_type+'.csv', index=False)  
rhythm_21days.to_csv(save_path + 'rhythm21days'+feat_type+'.csv', index=False)  

In [81]:
# grouped = data7.groupby(by=['participant_id'])
# columns = ['avg_hr', 'std_hr', 'Sleep Stage Sum']
# rhythm_7days = []
# for group in grouped:
    
#     df = copy.deepcopy(group[1])
#     ID = group[0]
#     curr_fea_df = []
    
#     for col_val in columns:
        
#         curr_window = copy.deepcopy(df[[col_val,'hour']])
        
#         M10H = M10(curr_window, col_val, 'hour')
#         L5H = L5(curr_window, col_val, 'hour')
        
#         AMP = (M10H-L5H)/(M10H+L5H)
        
#         DVT = deviation_from_template(curr_window, col_val, 'hour')
        
#         period_list = [8,16,20, 22, 27, 28, 32, 36, 64, 72, 128]
        
#         PSD = power_spectral_density(curr_window[col_val], period_list)
        
#         cosinor_value = fit_cosinor(curr_window, col_val, 'hour', period_list) 
        
#         fea_dict = {'M10':[M10H],'L5':[L5H], 'AMP':[AMP]}
        
#         fea_dict.update(PSD)
        
#         fea_dict.update(cosinor_value)
        
#         fea_dict2 = {col_val+'_'+k: v for k,v in fea_dict.items()}
        
#         DVT = DVT.mean().to_dict()
        
#         fea_df = pd.DataFrame(data={**fea_dict2, **DVT})
        
#         # fea_df = pd.concat((fea_df, DVT), axis=1)
        
#         if len(curr_fea_df) == 0:
#             curr_fea_df = fea_df
#         else:
#             curr_fea_df = pd.concat((curr_fea_df,fea_df), axis=1)
#     curr_fea_df['ID'] = ID
    
#     if len(rhythm_7days) == 0:
#         rhythm_7days = curr_fea_df
#     else:
#         rhythm_7days = pd.concat((rhythm_7days,curr_fea_df), axis=0) 
# rhythm_7days.reset_index(inplace=True, drop=True)